In [ ]:
# default_exp polygon_canvas

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from nbdev import *

In [ ]:
# exporti
from collections import deque
import numpy as np
import traitlets
import ipywidgets as widgets
from pathlib import Path
from math import log, sqrt, atan2, pi
from ipyevents import Event
from ipycanvas import MultiCanvas, hold_canvas
from ipywidgets import Image, Label, Layout, HBox, VBox, Output
from abc import ABC, abstractmethod
from ipycanvas import Canvas
from ipyannotator.bbox_canvas import draw_bg
from random import randrange

# Polygon Algorithms Helpers

### Convex Hull

The following class implements the Monothone Chain Algorithm to get the boundaries from a set of points, the present algorithm was inspired from [the wikibooks](https://en.wikibooks.org/wiki/Algorithm_Implementation/Geometry/Convex_hull/Monotone_chain#Python)

In [ ]:
# exporti
class ConvexHull:
    def _cross(self, o, a, b):
        return (a[0] - o[0]) * (b[1] - o[1]) - (a[1] - o[1]) * (b[0] - o[0])
    
    def execute(self, points, k):
        points = sorted(set(points))

        if len(points) <= 1:
            return points

        lower = []
        for p in points:
            while len(lower) >= 2 and self._cross(lower[-2], lower[-1], p) <= 0:
                lower.pop()
            lower.append(p)

        upper = []
        for p in reversed(points):
            while len(upper) >= 2 and self._cross(upper[-2], upper[-1], p) <= 0:
                upper.pop()
            upper.append(p)

        return lower[:-1] + upper[:-1]

### Concave Hull

The following class implements the Concave Hull Algorithm, based on Moreira and Santos (2007), to get the edge boundaries from a set of points. This particular algorithm is based on the Jarvi's March (gift wrapping) to calculate the concave hull, but uses a limited k-nearest neighbourhoods to do it. The code is similar to the paper.

#### Reference

Moreira, A. and Santos, M.Y., 2007, Concave Hull: A K-nearest neighbors approach for the computation of the region occupied by a set of points [PDF](https://repositorium.sdum.uminho.pt/bitstream/1822/6429/1/ConcaveHull_ACM_MYS.pdf)

In [ ]:
# exporti

class PointDistance:
    def _euclidian_distance(self, a, b):
        result = abs(a[0] - b[0])**2 + abs(a[1] - b[1])**2
        
        return sqrt(result)

class ConcaveHull(PointDistance):
    def _clean_list(self, points):
        return list(set(points))
    
    def _find_min_y_point(self, points):
        return min(points, key=lambda p: p[1])
    
    def _remove_point(self, points, point):
        points.remove(point)
        return points
    
    def _add_point(self, points, point):
        points.append(point)
        return points
    
    def _nearest_point(self, points, point, k):
        nearest = []
        for p in points:
            nearest.append(
                (self._euclidian_distance(point, p), p)
            )
        
        nearest = sorted(nearest, key=lambda p: p[0])
        result = []
        
        for i in range(min(k, len(points))):
            result.append(nearest[i][1])
        
        return result
            
    def _sort_by_angle(self, points, point, angle):
        """Sort in descending order of right-hand turn"""
        def compare(a):
            return self._angle_difference(angle, self._angle(point, a))
            
        return sorted(points, key=compare, reverse=True)

    def _intersects_q(self, line1, line2):
        a1 = line1[1][1] - line1[0][1]
        b1 = line1[0][0] - line1[1][0]
        c1 = a1 * line1[0][0] + b1 * line1[0][1]
        a2 = line2[1][1] - line2[0][1]
        b2 = line2[0][0] - line2[1][0]
        c2 = a2 * line2[0][0] + b2 * line2[0][1]
        tmp = (a1 * b2 - a2 * b1)
        if tmp == 0:
            return False
        sx = (c1 * b2 - c2 * b1) / tmp
        if (sx > line1[0][0] and sx > line1[1][0]) or (sx > line2[0][0] and sx > line2[1][0]) or\
                (sx < line1[0][0] and sx < line1[1][0]) or (sx < line2[0][0] and sx < line2[1][0]):
            return False
        sy = (a1 * c2 - a2 * c1) / tmp
        if (sy > line1[0][1] and sy > line1[1][1]) or (sy > line2[0][1] and sy > line2[1][1]) or\
                (sy < line1[0][1] and sy < line1[1][1]) or (sy < line2[0][1] and sy < line2[1][1]):
            return False
        return True
    
    def _angle(self, a, b):
        """Return the angle in radians between two points"""
        return atan2(b[1] - a[1], b[0] - a[0])
    
    def _angle_difference(self, a, b):
        if (a > 0 and b >= 0) and a > b:
            return abs(a - b)
        elif (a >= 0 and b > 0) and a < b:
            return 2 * pi + a - b
        elif ((a < 0 and b <= 0) and a < b):
            return 2 * pi + a + abs(b)
        elif ((a <= 0 and b < 0) and a > b):
            return abs(a - b)
        elif (a <= 0 and 0 < b):
            return 2 * pi + a - b
        elif (a >= 0 and 0 >= b):
            return a + abs(b)
    
        return 0.0
    
    def _point_in_polygon_q(self, point, points):
        x = point[0]
        y = point[1]
        poly = [(pt[0], pt[1]) for pt in points]
        n = len(poly)
        inside = False
        xints = 0.0

        p1x, p1y = poly[0]
        for i in range(n + 1):
            p2x, p2y = poly[i % n]
            if y > min(p1y, p2y):
                if y <= max(p1y, p2y):
                    if x <= max(p1x, p2x):
                        if p1y != p2y:
                            xints = (y - p1y) * (p2x - p1x) / (p2y - p1y) + p1x
                        if p1x == p2x or x <= xints:
                            inside = not inside
            p1x, p1y = p2x, p2y

        return inside
    
    def execute(self, points, k):
        kk = max(k, 2)
        dataset = self._clean_list(points)

        if len(dataset) < 3:
            return None
        
        if len(dataset) == 3:
            return dataset
        
        kk = min(kk, len(dataset))
        first_point = self._find_min_y_point(dataset)
        hull = [first_point]
        current_point = first_point
        dataset = self._remove_point(dataset, first_point)
        prev_angle = pi
        step = 2
        
        while ((current_point != first_point) or step == 2) and len(dataset) > 0:
            if step == 5:
                dataset = self._add_point(dataset, first_point)
            
            k_nearest_point = self._nearest_point(dataset, current_point, kk)
            c_points = self._sort_by_angle(k_nearest_point, current_point, prev_angle)
            its = True
            i = -1

            while its is True and i < (len(c_points)-1):
                i += 1
                
                if c_points[i] == first_point:
                    last_point = 1
                else:
                    last_point = 0
                
                j   = 2
                its = False
                
                while (its is False) and (j < (len(hull) - last_point)):
                    its = self._intersects_q((hull[step-2], c_points[i]), (hull[step-2-j], hull[step-1-j]))
                    j  += 1
            
            if its is True:
                """All candidates intersect restart with a new k"""
                return self.execute(points, kk+1)
        
            current_point = c_points[i]
            hull = self._add_point(hull, current_point)
            prev_angle = self._angle(hull[step-1], hull[step-2])
            dataset = self._remove_point(dataset, current_point)
            step += 1
    
        all_inside = True
        i = len(dataset)-1
    
        """ check if all the given points are inside the computed polygon """
        while (all_inside is True) and i >= 0:
            all_inside = self._point_in_polygon_q(dataset[i], hull)
            i -= 1
    
        """Since at least one point is out of the computed polygon, try again with a higher number of neighbours """
        if all_inside is False:
            return self.execute(points, kk+1)
    
        return hull

## Testing 

The following methods tests the hull algorithms

### Concave Hull

In [ ]:
p = ConcaveHull()


assert p._euclidian_distance((1, 1), (3,4)), sqrt(13)

assert p._clean_list([(1,2), (1,2)]), (1,2)

assert p._clean_list([(2,7), (3,4), (5,6)]), (3,4)

assert p._nearest_point([(2,7), (3,4), (5,6)], (1,1), 1), (3,4)
    
assert p._angle_difference(pi, 2*pi), pi

assert p._angle((20,20), (40,40)), 0.7853981633974483
    
assert p._sort_by_angle([(1,2), (3,4), (2,1)], (1,1), pi), [(2, 1), (3, 4), (1, 2)]

assert p._remove_point([(1,2), (3,4)], (1,2)), [(3,4)]
    
assert p._add_point([(1,2)], (3,4)), [(1,2), (3,4)]

assert p.execute([(1,2), (1,1), (3,4), (2,1)], 3), [(1, 1), (2, 1), (3, 4), (1, 2)]

# Draw Polygon on Canvas

In [ ]:
#exporti

from typing import List

def draw_polygon(canvas, coords: List[tuple], color='blue', line_fill_color='white', line_width=None, clear=False):
    with hold_canvas(canvas):
        if clear:
            canvas.clear()
            
        line_width = line_width or log(canvas.height)/5

        canvas.line_width = line_width*3  # we will have 1 inner and 2 outer lines, so multiply by 3
        canvas.line_cap = 'round'
        
        canvas.stroke_style = color
        canvas.stroke_polygon(coords)

        canvas.stroke_style = line_fill_color
        canvas.line_width = line_width
        canvas.stroke_polygon(coords)

        
def draw_points(canvas, coords: List[tuple], color='white', size=2, clear=False):
    with hold_canvas(canvas):
        if clear:
            canvas.clear()
        
        canvas.fill_style = color
        for point in coords:
            if point:
                canvas.fill_circle(point[0], point[1], size)

## Drawing with concave hull

The following section will show up draw using the concave hull

In [ ]:
### Ploting a five points concave hull
p = ConcaveHull()

points  = [(20, 20), (80, 20), (45, 30), (50,70), (80, 100)]
polygon = p.execute(points, 3)

canvas  = Canvas(width=125, height=125)
draw_bg(canvas)

draw_polygon(canvas, polygon, color='red', line_width=1.5)
draw_points(canvas, points, color='blue', size=5)

canvas

In [ ]:
### Ploting n points concave hull

n      = 1000
points = []

for i in range(n):
    points.append((randrange(1,900), randrange(1,900)))

polygon = p.execute(points, 3)

canvas  = Canvas(width=1000, height=1000)
draw_bg(canvas)

draw_polygon(canvas, polygon, color='red', line_width=1.5)
draw_points(canvas, points, color='blue', size=5)

canvas

In [ ]:
# test how draw_polygon works
canvas = Canvas(width=125, height=125)
draw_bg(canvas)

draw_polygon(canvas, [(10, 40), (10, 100)], color='black', line_width=2)
draw_polygon(canvas, np.array([(20, 20), (80, 20), (80, 100)]), color='red', line_width=1.5)
draw_polygon(canvas, np.array([(10, 10), (90, 10), (90, 110), (60, 110)]), color='green', line_width=2)
canvas

In [ ]:
# test how draw_polygon works
canvas = Canvas(width=125, height=125)
draw_bg(canvas)

draw_polygon(canvas, [(20, 20), (80, 20), (80, 100)], color='red', line_width=1.5)
draw_points(canvas, [(20, 20), (80, 20), (80, 100)], color='blue', size=5)

canvas

In [ ]:
# testing how capture the delete keyboard 

# from IPython.display import display
# from ipywidgets import Label, HTML, HBox, Image, VBox, Box, HBox


# canvas = Canvas(width=125, height=125)
# draw_bg(canvas)
# draw_polygon(canvas, [(20, 20), (80, 20), (80, 100)], color='red', line_width=1.5)

# d = Event(source=canvas, watched_events=['keydown'])

# def handle_event(event):
#     draw_points(canvas, [(10, 10)], color='blue', size=5)

# d.on_dom_event(handle_event)
# canvas

In [ ]:
from IPython.display import display
from ipywidgets import Label, HTML, HBox, Image, VBox, Box, HBox


canvas = Canvas(width=125, height=125)
draw_bg(canvas)
draw_polygon(canvas, [(20, 20), (80, 20), (80, 100)], color='red', line_width=1.5)


h = HTML('Event info')
d = Event(source=canvas, watched_events=['keydown'])

def handle_event(event):
    lines = {k:v for k, v in event.items()}
    content = str(lines['key'])
    h.value = content
    print(content)

d.on_dom_event(handle_event)
canvas

display(canvas, h)

# Create PolygoneCanvas

You can interactively use the widget in the following ways:

- Click in the desired location to add a new point to a polygon

- Press "Delete" or "Backspace" to remove the last point

- Press "Space" to stop drawing and adding new points

- Press "LeftArrow" or "RightArrow" to stop change last added point

- Move a cursor away from a widget to stop drawing and adding new points


In [ ]:
#export
from ipyannotator.bbox_canvas import draw_bounding_box, draw_img, get_image_size, points2bbox_coords
from IPython.display import display
from ipywidgets import Label, HTML, HBox, Image, VBox, Box, HBox


class PolygonCanvas(HBox, PointDistance, traitlets.HasTraits):
    """
    Represents canvas holding image and a polygon ontop. 
    Gives user an ability to draw a polygon with mouse.
    
    """
    debug_output = widgets.Output(layout={'border': '1px solid black'})
    image_path = traitlets.Unicode()
    _canvas_polygon_coords = traitlets.List()
    _image_scale = traitlets.Float()
    
    def __init__(self, width, height, k = 3):
        super().__init__()
        
        # used by the hull algorithm
        self._k = k
        self._hull = ConcaveHull()

        self._is_drawing = False
        self._current_point = None
        self._image_scale = 1.0
        
        self._bg_layer = 0
        self._image_layer = 1
        self._box_layer = 2
        # do not stick bbox to borders
        self.padding = 2

        # Define each of the children...
        self._image = Image(layout=Layout(display='flex',
                                          justify_content='center',
                                          align_items='center',
                                          align_content='center'))
        self._multi_canvas = MultiCanvas(3, width=width, height=height)
        self._im_name_box = Label()
            
        children = [VBox([self._multi_canvas, self._im_name_box])]
        self.children = children
        
        draw_bg(self._multi_canvas[self._bg_layer])
        
        self.d = Event(source=self, watched_events=['keydown'])
        
        # link drawing events        
        self._multi_canvas[self._box_layer].on_mouse_move(self._update_pos)
        self._multi_canvas[self._box_layer].on_mouse_up(self._add_point)
        self._multi_canvas[self._box_layer].on_mouse_out(self._stop_drawing)
        self.d.on_dom_event(self._del_nearest_point)
        
    @debug_output.capture(clear_output=False)            
    def _add_point(self, x, y):
#         print("-> ADD POINT")
        self._is_drawing = True
        self._canvas_polygon_coords.append((x, y))
        self._draw_polygon()
#         print("<- ADD POINT")
        
    def _remove_point(self, point):
        self._is_drawing = True
        self._canvas_polygon_coords.remove(point)
        self._draw_polygon()
    
    @debug_output.capture(clear_output=False)
    def _update_pos(self, x, y):
#         print("-> UPDATE POS")
        self._is_drawing = True
        self._current_point = (x, y)
        self._draw_polygon()
#         print("<- UPDATE POS")
   
    @debug_output.capture(clear_output=False)
    def _stop_drawing(self, x, y):
#         print("-> STOP DRAWING")
        self._is_drawing = False
        self._draw_polygon()
#         print("<- STOP DRAWING")

    @debug_output.capture(clear_output=False)
    @traitlets.observe('_canvas_polygon_coords', '_current_point')
    def _draw_polygon(self, change=None):
#         print('-> Observe _canvas_polygon_coords: ')  
        if not self._canvas_polygon_coords:
            self._clear_polygon()
            self._canvas_polygon_coords = []

            return

        if self._is_drawing:
            coords = (self._canvas_polygon_coords + [self._current_point])
        else: 
            coords = self._canvas_polygon_coords
        
        if len(coords) > 3:
            alpha_coords = self._hull.execute(coords, self._k)

            draw_polygon(self._multi_canvas[self._box_layer], alpha_coords, 
                         color='black', clear=True, line_width=1.5)
        else:
            draw_polygon(self._multi_canvas[self._box_layer], [], 
                         color='black', clear=True, line_width=1.5)
        
        draw_points(self._multi_canvas[self._box_layer], coords, 
                    color='green', size=3)
        
        self._select_nearest_point()
#         print('<- Observe canvas_coords')
        
    def _nearest_point(self):
        coords = self._canvas_polygon_coords
        nearest_point = None
        nearest_distance = None
        
        for coord in coords:
            distance = self._euclidian_distance(coord, self._current_point)
            
            if nearest_distance is None or nearest_distance > distance:
                nearest_distance = distance
                nearest_point = coord

        return nearest_point
    
    def _select_nearest_point(self):
        nearest = self._nearest_point()

        if nearest:
            draw_points(self._multi_canvas[self._box_layer], [nearest], 
                    color='yellow', size=3)
                
    def _del_nearest_point(self, event):
        event_as_dict = {k:v for k, v in event.items()}
        
        if event_as_dict['key'] == 'Delete':
            nearest = self._nearest_point()

            if nearest:
                self._remove_point(nearest)

    def _clear_polygon(self):
        self._multi_canvas[self._box_layer].clear()
        
    @traitlets.observe('image_path')
    def _draw_image(self, image):
        self._image_scale = draw_img(self._multi_canvas[self._image_layer], self.image_path, clear=True)
        self._im_name_box.value = Path(self.image_path).name

    @property
    def image_scale(self):
        return self._image_scale
    
    def _clear_image(self):
        self._multi_canvas[self._image_layer].clear()
    
    # needed to support voila
    # https://ipycanvas.readthedocs.io/en/latest/advanced.html#ipycanvas-in-voila
    def observe_client_ready(self, cb=None):
        self._multi_canvas.on_client_ready(cb)

In [ ]:
gui = PolygonCanvas(width=250, height=250)
# gui.image_path = '../data/projects/im2im1/class_images/blocks_1.png'

gui

In [ ]:
gui.debug_output

In [ ]:
gui._canvas_polygon_coords

In [ ]:
gui._canvas_polygon_coords = [(137.5625, 74.75), (44.5625, 167.75), (312.5625, 250.75)]

In [ ]:
gui.image_path = '../data/projects/im2im1/class_images/blocks_1.png'

In [ ]:
gui.image_scale

In [ ]:
gui._clear_polygon()

In [ ]:
#hide
from nbdev.export import *
notebook2script()

## User interaction with the canvas

As is right now

<!-- 
@startuml
actor User #white
control Canvas

loop for each pointer move
    User->Canvas: _on_mouse_move
    Canvas -> frontend: _update_pos
    frontend -> frontend: _draw_polygon
end

loop for each user click
    User -> Canvas: _on_mouse_up
    Canvas -> frontend : _add_point
    frontend -> frontend: _draw_polygon
end

User -> Canvas: _on_mouse_out
Canvas -> frontend: _stop_drawing
@enduml
-->

![](http://www.plantuml.com/plantuml/png/bOynReOm38Ltdy9Iv_y27H1Ipz0vif80KPCuIXnGRry34WCjGrVxy_FtnYPKfQS8P8KhVZPVyMrRWdYmdALon0_AApM0o5nmKiYJNR1moA9mujK38Xwdh-64tz5mDebxy-O2pXM-1fmgwsrsYlNYIBmft7RcsjmeLsbJ9dxFd457Tvc-QziOxDUbiYVybkdbGGML8kVCKUj_Ai_Vk0lysRe9boCfv1b6dVKKVm00)